In [57]:
# Import the dependencies.
import csv
import pandas as pd
import numpy as np
import requests
import time
import gmaps

# Import the API key.
from config import gmap_api_key

In [58]:
# Open the CSV created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Great Falls,US,2020-02-21 07:12:14,47.50,-111.30,41.00,23,75,25.28,broken clouds,0.00,0.0
1,1,Hithadhoo,MV,2020-02-21 07:12:14,-0.60,73.08,84.24,77,7,14.45,light rain,0.44,0.0
2,2,Necochea,AR,2020-02-21 07:12:14,-38.55,-58.74,57.99,64,62,21.99,light rain,0.19,0.0
3,3,Kodiak,US,2020-02-21 07:12:14,57.79,-152.41,32.00,74,90,10.29,overcast clouds,0.00,0.0
4,4,Bambous Virieux,MU,2020-02-21 07:12:14,-20.34,57.76,84.20,58,20,11.41,few clouds,0.00,0.0


In [63]:
# Ask the customer to add a minimum and maximum temperature value and preference for rain/snow.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 40
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) yes


In [64]:
# Filter for minimum and maximum temperature
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                     (city_data_df["Max Temp"] <= max_temp)]
# Filter for rain
if rain == "no":
    preferred_city_df = preferred_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] == 0)]
else:
    preferred_city_df = preferred_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] != 0)]

# Filter for snow                                        
if snow == "no":
    preferred_city_df = preferred_city_df.loc[(city_data_df["Snow inches (last 3 hrs)"] == 0)]
else:
    preferred_city_df = preferred_city_df.loc[(city_data_df["Snow inches (last 3 hrs)"] != 0)]

preferred_city_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
130,130,Nayoro,JP,2020-02-21 07:12:50,44.35,142.46,20.71,90,100,9.69,light snow,0.0,0.38
169,169,Grindavik,IS,2020-02-21 07:13:01,63.84,-22.43,32.00,57,100,21.92,light snow,0.0,0.44
193,193,Rudnichnyy,RU,2020-02-21 07:13:08,59.62,52.47,29.61,97,93,3.96,light snow,0.0,0.25
235,235,Teya,RU,2020-02-21 07:13:22,60.38,92.63,23.47,90,95,4.83,light snow,0.0,0.13
244,244,Gayny,RU,2020-02-21 07:13:24,60.31,54.32,28.74,93,99,5.53,light snow,0.0,0.38
326,326,Turukhansk,RU,2020-02-21 07:13:47,65.82,87.98,26.65,96,93,2.77,light snow,0.0,0.13
349,349,Novaya Igirma,RU,2020-02-21 07:13:52,57.13,103.91,24.76,88,100,9.91,light snow,0.0,0.19
351,351,Baykit,RU,2020-02-21 07:13:53,61.67,96.37,20.35,91,100,4.43,light snow,0.0,0.13
354,354,Yerbogachen,RU,2020-02-21 07:13:53,61.28,108.01,22.59,93,100,12.68,light snow,0.0,0.69
389,389,Yeniseysk,RU,2020-02-21 07:14:01,58.45,92.17,26.38,91,100,6.76,light snow,0.0,0.56


In [65]:
# Remove if any missing values
preferred_city_df = preferred_city_df.dropna()

In [66]:
# New dataframe for hotel data
hotel_df = preferred_city_df[["City", "Country", "Current Description","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
130,Nayoro,JP,light snow,20.71,44.35,142.46,
169,Grindavik,IS,light snow,32.00,63.84,-22.43,
193,Rudnichnyy,RU,light snow,29.61,59.62,52.47,
235,Teya,RU,light snow,23.47,60.38,92.63,
244,Gayny,RU,light snow,28.74,60.31,54.32,
326,Turukhansk,RU,light snow,26.65,65.82,87.98,
349,Novaya Igirma,RU,light snow,24.76,57.13,103.91,
351,Baykit,RU,light snow,20.35,61.67,96.37,
354,Yerbogachen,RU,light snow,22.59,61.28,108.01,
389,Yeniseysk,RU,light snow,26.38,58.45,92.17,


In [67]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmap_api_key
}

In [68]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gmap_api_key)

In [69]:
# Iterate through the DataFrame to get hotel data
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found....skipping.")

hotel_df.head(10)

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
130,Nayoro,JP,light snow,20.71,44.35,142.46,大雪荘
169,Grindavik,IS,light snow,32.00,63.84,-22.43,Northern Light Inn & Max's Restaurant
193,Rudnichnyy,RU,light snow,29.61,59.62,52.47,
235,Teya,RU,light snow,23.47,60.38,92.63,
244,Gayny,RU,light snow,28.74,60.31,54.32,
326,Turukhansk,RU,light snow,26.65,65.82,87.98,"ООО ""Туруханская пушнина"""
349,Novaya Igirma,RU,light snow,24.76,57.13,103.91,Tayga
351,Baykit,RU,light snow,20.35,61.67,96.37,Gostinitsa
354,Yerbogachen,RU,light snow,22.59,61.28,108.01,
389,Yeniseysk,RU,light snow,26.38,58.45,92.17,"""Domino"""


In [70]:
# Add a marker for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [71]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [72]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [73]:
# Add a marker for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [74]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")